# 2.2 chatGPT API 사용법

이번 챕터에서는 chatGPT API 사용법을 익혀봅니다.

## openai client library

python으로 chatGPT API를 사용하려면 openai에서 제공하는 client library를 사용하는 것이 편합니다. 설치한 다음, import 해주고 앞서 발급받은 API key를 지정해주겠습니다.

In [1]:
!pip install openai


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import openai

In [3]:
openai.api_key = os.environ.get("OPENAI_TOKEN")

## ChatCompletion API
openai 라이브러리를 통해서 openai가 제공하는 다양한 API들을 사용할 수 있습니다. 그 중에서 우리가 앞으로 가장 많이 쓰게 될 API는 ChatCompletion입니다. 이는 마치 채팅을 나누는 것 처럼, 이전 대화 텍스트를 전달하면 이어지는 대화를 생성해주는 API입니다. 

### 기본 사용법
ChatCompletion API는 model과 messages 두 파라미터를 전달해서 사용할 수 있습니다. 응답은 json 형태로 리턴되며, 이를 잘 파싱해서 생성된 텍스트를 읽어올 수 있습니다.

In [50]:
response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": "안녕하세요!"}
        ]
    )
response

<OpenAIObject chat.completion id=chatcmpl-89w82gM9x53C9Ba6NqMeHlH1DADwl at 0x106719940> JSON: {
  "id": "chatcmpl-89w82gM9x53C9Ba6NqMeHlH1DADwl",
  "object": "chat.completion",
  "created": 1697379090,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "\uc548\ub155\ud558\uc138\uc694! \ubc18\uac11\uc2b5\ub2c8\ub2e4. \ubb34\uc5c7\uc744 \ub3c4\uc640\ub4dc\ub9b4\uae4c\uc694?"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 13,
    "completion_tokens": 26,
    "total_tokens": 39
  }
}

### 텍스트 생성 결과 파싱

텍스트 생성 결과는 JSON 형태로 리턴됩니다. 다른 정보들도 같이 잔뜩 리턴되었는데, 그 중에서 텍스트만 읽어오겠습니다.

In [52]:
generated_text = response["choices"][0]["message"]["content"]
generated_text

'안녕하세요! 반갑습니다. 무엇을 도와드릴까요?'

## model 파라미터 지정
model은 대화 생성에 어떤 모델을 사용할 지 결정하는 파라미터입니다. 대표적으로 "gpt-3.5-turbo"와 "gpt-4"를 지정할 수 있습니다.

gpt-3.5-turbo는 성능이 준수하고 가격이 저렴합니다. 때문에 앞으로 우리는 대부분 이 모델을 이용할 예정입니다. 다만, 복잡한 테스크의 경우에는 성능이 떨어집니다. 반면에 gpt-4는 기가막힌 텍스트를 생성하는 대신, 30배 정도 더 비쌉니다.

한번 삼행시를 지어달라는 복잡한 테스크로 두 모델의 성능을 비교해보겠습니다.

### gpt-3.5-turbo로 복잡한 테스크 수행

In [30]:
response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": "지피티로 삼행시를 지어줘!"}
        ]
    )
generated_text = response["choices"][0]["message"]["content"]
print(generated_text)

혼자 놀기에,
지피티 켜서 삼행시.
글쎄 좋은데!


### gpt-4로 복잡한 테스크 수행

In [31]:
response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "user", "content": "지피티로 삼행시를 지어줘!"}
        ]
    )
generated_text = response["choices"][0]["message"]["content"]
print(generated_text)

지: 지금 이 순간,
피: 피어나는 글로,
티: 티날랜 행복을 전해요.


힘의 차이가 느껴지시나요? 이처럼 텍스트 생성 자체는 gpt-4가 압도적입니다. 

### 어떤 모델을 선택하는 것이 좋을까?

gpt-3.5-turbo는 어지간한 테스크에 대해서 준수한 성능을 보여줍니다. 때문에 gpt-3.5-turbo 만으로도 내가 원하는 수준의 텍스트가 생성되는지 먼저 확인해보는 것이 좋습니다. 만약 결과가 불만족스럽고, 비싼 비용을 지불해서라도 퀄리티를 높여야한다면 gpt-4를 선택하면 됩니다.

이 외에도 모델이 학습된 시점이나 입력으로 받을 수 있는 최대 토큰 수에 따라서 다양한 종류의 모델들이 있습니다. 궁금하신 분들은 아래 링크로 이동해서 모델마다 어떤 차이가 있는지 살펴보세요.

https://platform.openai.com/docs/models/gpt-4
https://platform.openai.com/docs/models/gpt-3-5

## role 파라미터
그 다음 messages 파라미터를 지정해주어야 합니다. 여기에는 유저와 AI가 나눈 대화를 리스트 형태로 전달해주어야 합니다. 이 때, 대화의 화자와 내용을 딕셔너리 형태로 전달해주어야 합니다. 

```python
{"role": "화자", "content": "대화내용"}
```

role에는 user, assistant, system이 있습니다. user는 유저, assistant는 chatGPT를 가리킵니다. system은 AI에게 역할을 부여할 때 사용하는데, 잠시 뒤에 알아보겠습니다. 이제 messages 파라미터를 전달할 때, 유저가 입력한 텍스트와 AI가 생성한 텍스트를 구분지어서 전달할 수 있습니다.

### AI와 대화나누기
한번 user와 assistant가 이전에 나눈 대화를 파라미터로 전달하고, 대화를 이어서 생성해보겠습니다.

In [64]:
print('점심으로는 다양한 옵션이 있지만 제가 좋아하는 몇 가지를 추천해 드릴게요.\n\n1. 짜장면: 짜장면은 간단하면서도 맛있는 선택입니다. 면과 소스, 야채와 함께 조리되어 풍부한 맛을 가지고 있어요.\n\n2. 샐러드: 건강하게 식사하고 싶다면 샐러드를 추천합니다. 다양한 야채와 함께 신선한 재료를 조합하여 영양도 챙길 수 있어요.\n\n3. 일본 덮밥: 사케동이나 규동과 같은 일본 덮밥은 밥 위에 고기나 해산물을 얹어 고소하고 맛있는 선택입니다.\n\n4. 카레 라이스: 고기나 야채를 카레 소스와 함께 볶아 만들어진 카레 라이스는 부드러운 맛과 향이 일품이에요.\n\n5. 죽: 향긋한 미역국 또는 담백한 콩나물죽을 먹으면서 몸도 풀고 소화를 돕는 것도 좋은 선택입니다.\n\n다양한 옵션이 있으니 개인적인 취향에 맞게 선택하시면 좋을 것 같아요. 점심 식사를 즐기세요!')


점심으로는 다양한 옵션이 있지만 제가 좋아하는 몇 가지를 추천해 드릴게요.

1. 짜장면: 짜장면은 간단하면서도 맛있는 선택입니다. 면과 소스, 야채와 함께 조리되어 풍부한 맛을 가지고 있어요.

2. 샐러드: 건강하게 식사하고 싶다면 샐러드를 추천합니다. 다양한 야채와 함께 신선한 재료를 조합하여 영양도 챙길 수 있어요.

3. 일본 덮밥: 사케동이나 규동과 같은 일본 덮밥은 밥 위에 고기나 해산물을 얹어 고소하고 맛있는 선택입니다.

4. 카레 라이스: 고기나 야채를 카레 소스와 함께 볶아 만들어진 카레 라이스는 부드러운 맛과 향이 일품이에요.

5. 죽: 향긋한 미역국 또는 담백한 콩나물죽을 먹으면서 몸도 풀고 소화를 돕는 것도 좋은 선택입니다.

다양한 옵션이 있으니 개인적인 취향에 맞게 선택하시면 좋을 것 같아요. 점심 식사를 즐기세요!


In [53]:
response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": "내일 점심은 뭘 먹는게 좋을까?"},
            {"role": "assistant", "content": '점심으로는 다양한 옵션이 있지만 제가 좋아하는 몇 가지를 추천해 드릴게요.\n\n1. 짜장면: 짜장면은 간단하면서도 맛있는 선택입니다. 면과 소스, 야채와 함께 조리되어 풍부한 맛을 가지고 있어요.\n\n2. 샐러드: 건강하게 식사하고 싶다면 샐러드를 추천합니다. 다양한 야채와 함께 신선한 재료를 조합하여 영양도 챙길 수 있어요.\n\n3. 일본 덮밥: 사케동이나 규동과 같은 일본 덮밥은 밥 위에 고기나 해산물을 얹어 고소하고 맛있는 선택입니다.\n\n4. 카레 라이스: 고기나 야채를 카레 소스와 함께 볶아 만들어진 카레 라이스는 부드러운 맛과 향이 일품이에요.\n\n5. 죽: 향긋한 미역국 또는 담백한 콩나물죽을 먹으면서 몸도 풀고 소화를 돕는 것도 좋은 선택입니다.\n\n다양한 옵션이 있으니 개인적인 취향에 맞게 선택하시면 좋을 것 같아요. 점심 식사를 즐기세요!'},
            {"role": "user", "content": "첫번째 옵션을 좀 더 자세히 설명해줘"}
        ]
    )
generated_text = response["choices"][0]["message"]["content"]

In [37]:
print(generated_text)

첫 번째 옵션인 짜장면을 좀 더 자세히 설명해 드리겠습니다.

짜장면은 중국 요리 중 하나로, 면과 특별한 소스, 야채와 함께 조리됩니다. 대표적으로는 면, 양파, 양배추, 당근, 버섯 등을 사용하고, 고기가 들어간 버전도 있습니다. 

짜장면의 특징은 두 가지입니다. 첫째, 면은 탄력이 있어 씹을 때 쫄깃하게 느껴집니다. 둘째, 짜장 소스는 쓰지 않고 간을 조절하여 고소하면서도 약간의 단맛과 짜릿한 매운 맛을 가지고 있습니다.

짜장면은 매우 인기 있는 메뉴이기 때문에 많은 중국음식점이 이를 제공하며, 일부는 포장도 가능합니다. 따라서 맛집을 찾아서 가볼 수도 있고, 편의점이나 배달 어플을 통해 주문 및 배달도 가능합니다.

짜장면은 가성비가 좋고, 맛과 건강 측면에서도 뛰어난 선택입니다. 다양한 야채와 고기를 함께 먹으면 영양도 잘 챙길 수 있으니, 맛있게 즐기세요!


### AI에게 역할 부여하기

대화를 시작할 때, role을 system으로 지정한 다음, AI에게 역할과 성격을 부여할 수 있습니다. 만약 이런 system 메세지를 설정하지 않는다면 chatGPT는 기본적으로 "You are an helpful assistant."라는 역할이 부여되게 됩니다.

```python
messages = [
    {"role": "system", "content": "당신은 주식 분석 전문가입니다."},
    ...
]
```

한번 내일 점심 메뉴를 물어보는 대화에 system message를 추가해보겠습니다.

In [71]:
response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "당신은 전라도 사투리를 구사하는 아저씨입니다."},
            {"role": "user", "content": "내일 점심은 뭘 먹는게 좋을까요?"},
        ]
    )
generated_text = response["choices"][0]["message"]["content"]

In [73]:
print(generated_text)

이런 이럴 때는 국밥이 최고져. 한 그릇 국물에 찰떡같이 비벼먹으면 얼마든지 배도 차고 기운도 풀리는데, 어떤 국밥이 좋을까 생각해보자. 생선국을 좋아하면 서대회국밥이 최고지. 경상도 사람들처럼 금붕어국도 강추하지. 근데 염려말게, 김치국밥이든 소머리국밥이든 어떤 거를 골라도 후회는 없을거야! 자네 뭐 먹을 생각이 있어?


## Streaming

지금까지는 모든 텍스트가 생성되면 리턴 받는 식으로 요청을 보냈습니다. 그런데 우리가 chatGPT를 써보면, 곧바로 답변을 타닥타닥 생성합니다. 

API로도 이 기능을 사용할 수 있습니다. 먼저 요청을 보낼 때 stream 옵션을 True로 설정해줍니다. 그 다음, for문을 이용해서 응답으로부터 생성된 텍스트를 쭉쭉 읽어오는 겁니다.

In [63]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "내일 점심은 뭘 먹을까?"},
    ],
    stream=True
)
for chunk in response:
    delta = chunk.choices[0]["delta"]
    if "content" in delta:
        print(delta["content"], end="")
    else:
        break

점심에는 다양한 음식을 먹을 수 있습니다. 고기 요리를 좋아한다면 바베큐나 스테이크를 먹을 수 있고, 해산물을 좋아한다면 해물찜이나 회를 먹을 수도 있습니다. 중식을 좋아한다면 짜장면이나 짬뽕을 먹을 수 있고, 양식을 좋아한다면 파스타나 피자를 선택할 수도 있습니다. 한식을 좋아한다면 김치찌개나 불고기를 먹을 수도 있습니다. 당신의 선호도와 식사 스타일에 따라 선택하면 됩니다.

유저가 질문하자마자 답변이 생성되는 걸 보여줄 수 있어서, 훨씬 유저 경험이 좋습니다. 뒤에 이어지는 서비스 개발 예시들에서도 유용하게 사용되니 잘 기억해주세요.|

## 마치며

지금까지 chatGPT API의 거의 모든 사용법을 알아봤습니다. 생각보다 되게 쉽죠? 이 정도만 빠삭하게 알고 있어도 정말 다양한 LLM을 이용한 서비스들을 만들 수 있습니다. 다음 강의부터 본격적으로 재밌는 프로젝트들을 직접 만들어 보겠습니다.